# Dask Bag

Материалы: 
* Макрушин С.В. Лекция 12: Map-Reduce
* https://docs.dask.org/en/latest/bag.html
* Jesse C. Daniel. Data Science with Python and Dask. 

## Задачи для совместного разбора

In [1]:
import nltk

1. Считайте файл `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt` и разбейте на предложения. Подсчитайте длину (в кол-ве символов) каждого предложения.

In [2]:
text = open('Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt').read()
sents = nltk.sent_tokenize(text)
sents[:3]

['Спасибо, что скачали книгу в бесплатной электронной библиотеке BooksCafe.Net: http://bookscafe.net\n\nВсе книги автора: http://bookscafe.net/author/dostoevskiy_fedor-1096.html\n\nЭта же книга в других форматах: http://bookscafe.net/book/dostoevskiy_fedor-igrok-240117.html\n\nПриятного чтения!',
 'Федор Михайлович Достоевский\n\nИгрок\n\n(Из записок молодого человека)\n\n\n\n\nГлава I\n\nНаконец я возвратился из моей двухнедельной отлучки.',
 'Наши уже три дня как были в Рулетенбурге.']

In [3]:
[len(sent) for sent in sents][:3]

[287, 133, 41]

In [4]:
list(map(len, sents))[:3]

[287, 133, 41]

2. Считайте файл `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt` и разбейте на предложения. Выведите предложения, длина которых не более 10 символов.

In [5]:
[sent for sent in sents if len(sent) <= 10][:3]

['— ton).]', '—\xa0Плюнуть?', '«Как!']

In [7]:
list(filter(lambda sent: len(sent) <= 10, sents))[:3]

['— ton).]', '—\xa0Плюнуть?', '«Как!']

3. На основе списка предложений из задачи 1-2 создайте `dask.bag`. Рассчитайте среднюю длину предложений в тексте.

In [10]:
import dask.bag as db

In [15]:
bag = db.from_sequence(sents)
bag.map(len).mean().compute()

72.89840944073885

4. На основе файла `addres_book.json` создайте `dask.bag`. Посчитайте количество мобильных и рабочих телефонов в наборе данных

In [27]:
import json

In [28]:
data = json.load(open('addres-book.json'))
data[0]

{'name': 'Faina Lee',
 'email': 'faina@mail.ru',
 'birthday': '22.08.1994',
 'phones': [{'phone': '232-19-55'}, {'phone': '+7 (916) 232-19-55'}]}

In [29]:
bag = db.from_sequence(data)
bag

dask.bag<from_sequence, npartitions=8>

In [22]:
bag.map(lambda record: record['phones']).compute()

[[{'phone': '232-19-55'}, {'phone': '+7 (916) 232-19-55'}],
 [{'phone': '111-19-55'}],
 [{'phone': '232-19-56'}, {'phone': '+7 (916) 232-19-56'}],
 [{'phone': '+7 (916) 445-19-57'}],
 [{'phone': '232-19-58'}, {'phone': '+7 (916) 232-19-58'}],
 [{'phone': '+7 (916) 445-19-59'}],
 [{'phone': '232-19-50'}, {'phone': '+7 (916) 232-19-50'}],
 [{'phone': '111-19-51'}, {'phone': '+7 (916) 445-19-51'}]]

In [25]:
bag.pluck('phones').flatten().compute()

[{'phone': '232-19-55'},
 {'phone': '+7 (916) 232-19-55'},
 {'phone': '111-19-55'},
 {'phone': '232-19-56'},
 {'phone': '+7 (916) 232-19-56'},
 {'phone': '+7 (916) 445-19-57'},
 {'phone': '232-19-58'},
 {'phone': '+7 (916) 232-19-58'},
 {'phone': '+7 (916) 445-19-59'},
 {'phone': '232-19-50'},
 {'phone': '+7 (916) 232-19-50'},
 {'phone': '111-19-51'},
 {'phone': '+7 (916) 445-19-51'}]

In [30]:
def add_type(record):
    record['type'] = 'mobile' if record['phone'].startswith('+7') else 'work'
    return record

In [37]:
bag.pluck('phones').flatten().map(add_type).compute()

[{'phone': '232-19-55', 'type': 'work'},
 {'phone': '+7 (916) 232-19-55', 'type': 'mobile'},
 {'phone': '111-19-55', 'type': 'work'},
 {'phone': '232-19-56', 'type': 'work'},
 {'phone': '+7 (916) 232-19-56', 'type': 'mobile'},
 {'phone': '+7 (916) 445-19-57', 'type': 'mobile'},
 {'phone': '232-19-58', 'type': 'work'},
 {'phone': '+7 (916) 232-19-58', 'type': 'mobile'},
 {'phone': '+7 (916) 445-19-59', 'type': 'mobile'},
 {'phone': '232-19-50', 'type': 'work'},
 {'phone': '+7 (916) 232-19-50', 'type': 'mobile'},
 {'phone': '111-19-51', 'type': 'work'},
 {'phone': '+7 (916) 445-19-51', 'type': 'mobile'}]

In [31]:
bag.pluck('phones').flatten().map(
    lambda record: 'mobile' if record['phone'].startswith('+7') else 'work').frequencies().compute()

[('work', 6), ('mobile', 7)]

In [39]:
(bag.pluck('phones')
 .flatten()
 .map(add_type)
 .pluck('type')
 .frequencies()
 .compute())

[('work', 6), ('mobile', 7)]

In [40]:
phones_with_types = bag.pluck('phones').flatten().map(add_type)
type_freqs = phones_with_types.pluck('type').frequencies()
type_freqs.compute()

[('work', 6), ('mobile', 7)]

In [240]:
def binop(total, value):
    return total + 1


def combine(left, right):
    return left + right

In [242]:
bag.pluck('phones')\
.flatten()\
.map(add_type)\
.foldby('type', binop, 0, combine, 0)\
.compute()

[('work', 6), ('mobile', 7)]

## Лабораторная работа 12

In [1]:
import dask.bag as db
import json
import re
import pandas as pd

1. В файлах архиве `reviews_full.zip` находятся файлы, содержащие информацию об отзывах к рецептам в формате JSON Lines. Отзывы разделены на файлы в зависимости от оценки (например, в файле `reviews_1.json` находятся отзывы с оценкой 1). Считайте файлы из этого архива в виде `dask.bag`. Преобразуйте текстовое содержимое файлов с помощью модуля `json`. Выведите на экран первые 5 элементов полученного `bag`.

In [69]:
import os

fn = os.path.join("./reviews_full/reviews_*.json")
fn

'./reviews_full/reviews_*.json'

In [2]:
b = db.read_text("./reviews_full/reviews_*.json").map(json.loads)
b.take(5)

({'user_id': 452355,
  'recipe_id': 292657,
  'date': '2016-05-08',
  'review': 'WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.'},
 {'user_id': 329304,
  'recipe_id': 433404,
  'date': '2006-06-14',
  'review': 'This was good but the dressing needed something and I found it to be a little too sweet, next time I will experiment with some garlic and herbs and reduce the sugar slightly, thanks for sharing kcdlong!...Kitten'},
 {'user_id': 227932,
  'recipe_id': 2008187,
  'date': '1985-11-19',
  'review': 'Very good,it was a hit for my family. I used 6 cloves of garlic and had 1 lb beef and  Johnsonville sausage,1/2 lb hot and  1/2 lb honey garlic( which I wanted to use). That was a perfect combo for us. The sausage gave it nice flavor No guestion , I will be making this often.'},
 {'u

2. Модифицируйте функцию разбора JSON таким образом, чтобы в каждый словарь c информацией об отзыве добавить ключ `rating`. Значение получите на основе названия файла (см. аргумент `include_path`), использовав для этого регулярное выражение.

In [3]:
bag = db.read_text("./reviews_full/reviews_*.json", include_path=True)

In [15]:
def add_rating_column(string):
    import ast
    import re

    dct = ast.literal_eval(re.sub(r"\n", '', string[0]))
    dct['rating'] = int(re.search(r"\d.json", string[1]).group(0)[0])
    return dct

In [16]:
bag_with_rating = (bag.map(lambda x: add_rating_column(x))
                   .map(json.dumps)
                   .map(json.loads))

bag_with_rating.take(5)

({'user_id': 452355,
  'recipe_id': 292657,
  'date': '2016-05-08',
  'review': 'WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.',
  'rating': 0},
 {'user_id': 329304,
  'recipe_id': 433404,
  'date': '2006-06-14',
  'review': 'This was good but the dressing needed something and I found it to be a little too sweet, next time I will experiment with some garlic and herbs and reduce the sugar slightly, thanks for sharing kcdlong!...Kitten',
  'rating': 0},
 {'user_id': 227932,
  'recipe_id': 2008187,
  'date': '1985-11-19',
  'review': 'Very good,it was a hit for my family. I used 6 cloves of garlic and had 1 lb beef and  Johnsonville sausage,1/2 lb hot and  1/2 lb honey garlic( which I wanted to use). That was a perfect combo for us. The sausage gave it nice flavor No guestion , I will

3. Посчитайте количество отзывов в исходном датасете.

In [9]:
b = db.read_text("./reviews_full/reviews_*.json").map(json.loads)
b.count().compute()

9057540

4. Отфильтруйте `bag`, сохранив только отзывы, оставленные в 2014 и 2015 годах.

In [6]:
b_filtered_by_years = bag_with_rating.filter(lambda i: i['date'].split("-")[0] in ('2014', '2015'))
b_filtered_by_years

dask.bag<filter-lambda, npartitions=6>

In [14]:
b_filtered_by_years.count().compute()

735274

In [23]:
bag_with_rating.pluck('date').take(5)     #i['date'].split("-")[0] in ['2014', '2015'])

('2016-05-08', '2006-06-14', '1985-11-19', '2019-05-21', '1972-09-18')

In [11]:
b_filtered_by_years.take(50)

({'user_id': 229850,
  'recipe_id': 1300038,
  'date': '2014-10-03',
  'review': 'Took this to a New Year&#039;s Eve Party. Everyone loved it! It&#039;s absolutely perfect, the flavor, the crunch, just delicious!',
  'rating': '0'},
 {'user_id': 2706705,
  'recipe_id': 133747,
  'date': '2015-05-08',
  'review': 'Simple and easy way to enjoy a slice of pizza any time!  Well-toasted bread is the key - really toast it!  I put a bit of pizza sauce underneath my cheese for a more pizza-like flavor.  I used sourdough bread & medium cheddar cheese.  Fast & fun!  Great idea!  Made for 1-2-3 Hits Tag Game.',
  'rating': '0'},
 {'user_id': 945545,
  'recipe_id': 898468,
  'date': '2015-06-30',
  'review': 'Delish!  I wanted to make this spicy so I used hot enchilada sauce and jalapeno refried beans.  I forgot to buy the onions so I doctored up the beans with onion powder and granulated garlic.  Added the olives under the cheese and baked, uncovered, for the 25 minutes.  Served with pico de gall

5. Выполните препроцессинг отзывов:
    * привести строки к нижнему регистру
    * обрезать пробельные символы в начале и конце строки
    * удалите все символы, кроме английских букв и пробелов
    
Примените препроцессинг ко всем записям из `bag`, полученного в задании 4.

In [17]:
def preprocess(record):
    record["review"] = re.sub(r'[^A-Za-z ]+', '', record["review"].lower().strip())
    return record

In [18]:
b_filtered_and_preprocessed = b_filtered_by_years.map(lambda x: preprocess(x))
b_filtered_and_preprocessed.take(5)

({'user_id': 229850,
  'recipe_id': 1300038,
  'date': '2014-10-03',
  'review': 'took this to a new years eve party everyone loved it its absolutely perfect the flavor the crunch just delicious',
  'rating': 0},
 {'user_id': 2706705,
  'recipe_id': 133747,
  'date': '2015-05-08',
  'review': 'simple and easy way to enjoy a slice of pizza any time  welltoasted bread is the key  really toast it  i put a bit of pizza sauce underneath my cheese for a more pizzalike flavor  i used sourdough bread  medium cheddar cheese  fast  fun  great idea  made for  hits tag game',
  'rating': 0},
 {'user_id': 945545,
  'recipe_id': 898468,
  'date': '2015-06-30',
  'review': 'delish  i wanted to make this spicy so i used hot enchilada sauce and jalapeno refried beans  i forgot to buy the onions so i doctored up the beans with onion powder and granulated garlic  added the olives under the cheese and baked uncovered for the  minutes  served with pico de gallo sour cream and avocado chunks  fantastic  tha

6. Посчитайте количество отзывов в датасете, полученном в результате решения задачи 5. В случае ошибок прокомментируйте результат и исправьте функцию препроцессинга.

In [31]:
b_filtered_and_preprocessed.count().compute()

735274

7. Посчитайте, как часто в наборе, полученном в задании 5, встречается та или иная оценка.

In [21]:
dict(b_filtered_and_preprocessed.pluck("rating").frequencies())

{0: 42472, 1: 9246, 2: 9380, 3: 26532, 4: 119413, 5: 528231}

8. Найдите среднее значение `rating` в наборе, полученном в задании 5.

In [19]:
b_filtered_and_preprocessed.pluck('rating').mean().compute()

4.388036296673077

9. Используя метод `foldby`, подсчитать максимальную длину отзывов в зависимости от оценки `rating` в наборе, полученном в задании 5.

In [239]:
def binop(total, value):
    return max(total, len(value["review"]))

def combine(left, right):
    return max(left, right)

dict(b_filtered_and_preprocessed.foldby("rating", binop, 0, combine, 0))

{0: 6548, 1: 2868, 2: 2834, 3: 3174, 4: 6548, 5: 5343}